# Watershed Challenge

## Setup

In [ ]:
import pandas as pd
import numpy as np
import re
 
from statsmodels.tsa.seasonal import seasonal_decompose

import seaborn as sns
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
import matplotlib.dates as mdates
from matplotlib.lines import Line2D

pd.options.display.max_columns = 1100
pd.set_option('display.max_rows', 100)
pd.set_option('display.float_format', lambda x: '%.5f' % x)

sns.reset_defaults()
sns.set(
    rc={'figure.figsize':(7,4)}, 
    style="whitegrid", 
    palette=sns.color_palette("hls", 3)
)

## Overview

We will try to predict extreme watershed events in Chile.

Some of the key questions we could try to answer are:

- Has the frequency of heat waves events increased over the last years?
- Is there any relationship between heat waves and peak flow events?
- If so, can we correlate those events with the watershed's features?

## Data

In [ ]:
flux_data = pd.read_csv('flux.csv')

In [ ]:
flux_data.date = pd.to_datetime(flux_data.date, format='%Y-%m-%d')

In [ ]:
flux_data

### General Data Quality

Let us adjust the variables names to correspond to that in the challenge documentation.

In [ ]:
flux_data.rename(
    columns = {
        'basin_id':'station_code',
        'gauge_name':'station_name'
    },
    inplace = True)

In [ ]:
flux_data.describe(datetime_is_numeric=True)

In [ ]:
flux_data.nunique()

We see that there are `372` unique values for both `station_code` and `station_name`, so we discard any observational identification issues.

However, we see lower number of distinct values for `lat`, `lon`, `mean_elev` and `area_km2`. Given the fact that this is not a raw dataset and it's been already aggregated and enriched, we won't be inspecting this difference now due to time constraints, since it's doesn't seem relevant.

In [ ]:
flux_data[['station_code', 'station_name', 'lat', 'lon', 'mean_elev', 'area_km2']].drop_duplicates()

`Rio Tolten En Teodoro Schmidt` is the only station with two distinct values for `area_km2`, but the difference is negligible. We stick to the first one in order to keep a station reference dataset with `372` rows.

In [ ]:
station_data = flux_data.groupby(['station_code', 'station_name', 'lat', 'lon', 'mean_elev']).agg({
    'area_km2':max,
    'date':'nunique'
}).reset_index()

In [ ]:
station_data

In [ ]:
def river_name(x):
    
    river_name = re.search(r'(.*?)(En|Antes|A.)(.*?)$', x)
    
    if river_name:
        return river_name.group(1)
    else:
        return x
    
def place_name(x):    
    
    place_name = re.search(r'En (.*)', x)
    
    if place_name:
        return place_name.group(1)
    else:
        return ''

In [ ]:
station_data['river_name'] = station_data.station_name.apply(lambda x: river_name(x))
#station_data['place_name'] = station_data.station_name.apply(lambda x: place_name(x))

In [ ]:
station_data

In [ ]:
station_data.river_name.value_counts()

### Lat, lon

In [ ]:
ax = sns.scatterplot(data = station_data, 
                     y = 'lat', x = 'lon',
                     hue = 'mean_elev',)

plt.show()

### One TS

In [ ]:
flux_data

In [ ]:
def plot_one_timeserie(station_code, variable, min_date='1980-01-01', max_date='2023-01-01'):

    one_timeserie = flux_data[flux_data['station_code'] == station_code].set_index('date')[variable]

    sample = one_timeserie.loc[min_date:max_date]

    ax = sns.lineplot(data = sample)
    ax.set(title = f"{flux_data[flux_data.station_code == station_code]['station_name'].tolist()[0]}")
    ax.xaxis.set_major_formatter(mdates.DateFormatter('%b/%y'))

    plt.show()

#### Flux

In [ ]:
plot_one_timeserie(9433001, 'flux', '1990-01-01', '2020-02-03')

#### Precip

In [ ]:
plot_one_timeserie(1001001, 'precip', '2000-01-01', '2010-02-03')

#### Temp_max

In [ ]:
plot_one_timeserie(1001002, 'temp_max', '2005-01-01', '2010-02-03')

### Three TS

In [ ]:
def plot_three_timeseries(station_code, min_date='1980-01-01', max_date='2023-01-01'):

    three_timeseries = flux_data[flux_data.station_code == station_code].set_index('date')

    sample = three_timeseries.loc[min_date:max_date][['temp_max', 'precip', 'flux']]
    
    normalized_sample = (sample - sample.mean()) / sample.std()

    ax = sns.lineplot(data = normalized_sample)
    ax.set(title = f"{three_timeseries['station_name'].tolist()[0]}")
    ax.xaxis.set_major_formatter(mdates.DateFormatter('%b/%y'))

    sns.move_legend(ax, "upper left", bbox_to_anchor=(1, 1))

    plt.show()

In [ ]:
plot_three_timeseries(1001001, '2000-01-01', '2010-02-03')

In [ ]:
plot_three_timeseries(1001001, '2000-01-01', '2010-02-03')

## Extremes

We proceed with a working example for station `1001002`.

This first exercise takes into account the seasonality of the phenomena and only then we look at the percentiles to catch the days that were atypical, in an univariate sense, to identify these days as extreme ones.

In the following chunks, we use `statsmodels.tsa.seasonal.seasonal_decompose` to subtract the seasonal component of each series.
Eventual missing points are interpolated using `method='linear'`, and the decomposition is applied using the additive approach, for the sake of simplicity.

In [ ]:
single_ts = flux_data[flux_data.station_code == 1001002]\
.set_index('date')\
.asfreq('D')

In [ ]:
single_ts

In [ ]:
single_ts_in = flux_data[flux_data.station_code == 1001002]\
.set_index('date')\
.asfreq('D')\
.interpolate(method='linear')

#### Additive model

In [ ]:
result = seasonal_decompose(single_ts_in['flux'], model="additive", period = 365)

In [ ]:
plt = result.plot()

In [ ]:
result_values = pd.DataFrame([result.observed, result.seasonal, result.trend, result.resid]).T.dropna()
result_values['excess'] = result_values['flux'] - result_values['seasonal']
result_values.describe()

In [ ]:
extremes = result_values[result_values.excess > result_values.excess.quantile(0.95)]
extremes

In [ ]:
ax = sns.lineplot(data = result_values.excess)
ax.set(title = f"Seasonaly adjusted flux for 1001002 station")
ax.xaxis.set_major_formatter(mdates.DateFormatter('%b/%y'))
plt.show()

In [ ]:
ax = sns.lineplot(data = result_values.flux)
ax.set(title = f"Flux for 1001002 station")
ax.xaxis.set_major_formatter(mdates.DateFormatter('%b/%y'))
plt.show()

In [ ]:
ax = sns.lineplot(data = extremes.excess)
ax.set(title = f"Only extremes for 1001002 station")
ax.xaxis.set_major_formatter(mdates.DateFormatter('%b/%y'))
plt.show()

In [ ]:
extremes_index = extremes.index
extremes_index

### Conclusions

> This variables should take the value of 1 when that variable in a specific day was extreme. Being extreme could be considered as being greater than expected. For example, a flux can be considered as extreme (value 1) when is over the 95 percentile of the flux distribution for that specific season, and takes the value 0 otherwise. Taking into account the seasonality of that variables is very important, because  could be considered as extreme in wintertime, but it’d be a normal temperature for summertime.

1. Do you consider this a good way of capturing extreme events?

It's a fair way. It makes sense to account for seasonality. Since our context is global warming, we would welcome some approach that references directly the trend component, for instance. "Greater than expected" is subjective. If we consider that we shouldn't be expecting an underlying positive trend, looking for the deviation from the seasonal component fulfills the objective at first sight.

2. Or you would have used a different method? Which one?

Nevertheless, we can see from the original time series plot for the example above that excess flux are easily recognizable without any advanced statistical procedures. This raises a point on which aspect of the problem concerns us the most. A flux of 3 units in the dry season is as relevant as a flux of 20 units in the rainy season, given that both values represent a 0.95 percentile for that season? Is it a substancial occurrence?
Nature has ways to deal with these uncommon events, for instance, the ground absortion of water. So given two water sprouts events of the same magnitude, the one that happens when the soil is dry is less critical than the one that happens after a week or two of daily strong water sprouts, when the soil were already soaked.
The same could go for heat waves, once we could be concern with direct impact on human beings, so despite of seasonality being important, we could study to ignore it and only look for time-invariant variability.
Either ways, this invites an approach of moving averages for instance. And we could then naively look for extreme events (perc > 0.95) .

### Wrap-up

Let's turn this into a function and loop through all stations and variables.

In [ ]:
def get_extremes(station_code, variable):
    
    single_ts_in = flux_data[flux_data.station_code == station_code].set_index('date')\
    [variable]\
    .asfreq('D')\
    .interpolate(method='linear')
    
    result = seasonal_decompose(single_ts_in, model="additive", period=365)
    
    result_values = pd.DataFrame([result.observed, result.seasonal]).T.dropna()
    result_values['excess'] = result_values[variable] - result_values['seasonal']
    
    extremes = result_values[result_values.excess > result_values.excess.quantile(0.95)]
    
    return extremes.index

In [ ]:
flux_data.loc[:, 'flux_extreme'] = 0
flux_data.loc[:, 'temp_max_extreme'] = 0
flux_data.loc[:, 'precip_extreme'] = 0

In [ ]:
for variable in ['flux', 'temp_max', 'precip']:
    
    print(variable)
    
    for station_code in station_data[station_data.date > 730].reset_index()['station_code']:
                
        extremes = get_extremes(station_code, variable)
        
        flux_data.loc[
            (flux_data.station_code == station_code) & (flux_data.date.isin(extremes)), 
            f"{variable}_extreme"
        ] = 1

In [ ]:
flux_data[['flux_extreme','temp_max_extreme','precip_extreme']].isna().sum()

In [ ]:
#flux_data.to_csv('flux_data_rich.csv')

### Monthly

In [ ]:
monthly_mean = flux_data.resample(rule='M', on='date').mean()[['flux_extreme','temp_max_extreme','precip_extreme']]

In [ ]:
monthly_mean

In [ ]:
ax = sns.lineplot(data = monthly_mean.loc[:'2020-05-01'])
ax.set(title = f"ueh")
ax.xaxis.set_major_formatter(mdates.DateFormatter('%b/%y'))
sns.move_legend(ax, "upper left", bbox_to_anchor=(1, 1))
plt.show()

### Annualy

In [ ]:
annual_mean = flux_data.resample(rule='Y', on='date').mean()[['flux_extreme','temp_max_extreme','precip_extreme']]

In [ ]:
annual_mean

In [ ]:
ax = sns.lineplot(data = annual_mean.loc[:'2020-05-01'])
ax.set(title = f"ueh")
ax.xaxis.set_major_formatter(mdates.DateFormatter('%b/%y'))
sns.move_legend(ax, "upper left", bbox_to_anchor=(1, 1))
plt.show()

# Questions left to discuss

1. Are there any different behaviours among different watersheds?

2. Have they become more frequent?

3. Extreme flux prediction: modeling and discussion